In [1]:
import pandas as pd
import numpy as np

In [3]:
df_full = pd.read_parquet("stocks_df_combined_2024_05_07.parquet.brotli")
df_full.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,0.088542,0.101563,0.088542,0.097222,0.060163,1.031789e+09,MSFT,1986,1986-03-01,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.097222,0.102431,0.097222,0.100694,0.062311,3.081600e+08,MSFT,1986,1986-03-01,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.100694,0.103299,0.100694,0.102431,0.063386,1.331712e+08,MSFT,1986,1986-03-01,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.102431,0.103299,0.098958,0.099826,0.061774,6.776640e+07,MSFT,1986,1986-03-01,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.099826,0.100694,0.097222,0.098090,0.060700,4.789440e+07,MSFT,1986,1986-03-01,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Question 1

In [17]:
df_full["week_of_month"] = ((df_full["Date"].dt.day - 1) // 7) + 1
df_full["month_wm"] = df_full["Date"].dt.month_name().str.cat(df_full["week_of_month"].astype(str), sep="-")
df_full["month_wm"].head()

0    March-2
1    March-2
2    March-3
3    March-3
4    March-3
Name: month_wm, dtype: object

In [24]:
correlations = pd.get_dummies(df_full["month_wm"]).corrwith(df_full["is_positive_growth_5d_future"])
correlations.head()

April-1   -0.001049
April-2    0.002610
April-3    0.006504
April-4   -0.001498
April-5   -0.005997
dtype: float64

In [29]:
correlations[[correlations.abs().idxmax()]]

September-3   -0.030726
dtype: float64

## Question 2

In [51]:
df_full["pred3_manual_gdp_fastd"] = (df_full["gdppot_us_yoy"] <= 0.027) & (df_full["fastd"] >= 0.251)
df_full["pred4_manual_gpd_wti_oil"] = (df_full["gdppot_us_yoy"] >= 0.027) & (df_full["growth_wti_oil_30d"] <= 1.005)

acc_3 = (df_full["pred3_manual_gdp_fastd"] * df_full["is_positive_growth_5d_future"]).sum() / df_full["pred3_manual_gdp_fastd"].sum()
acc_4 = (df_full["pred4_manual_gpd_wti_oil"] * df_full["is_positive_growth_5d_future"]).sum() / df_full["pred4_manual_gpd_wti_oil"].sum()

acc_3, acc_4

(0.5500331156077285, 0.5374581350255596)